## Check NVIDIA GPU config

In [ ]:
!nvidia-smi

## Google Drive create environment

In [ ]:
from google.colab import drive

google_drive_environment = True

if google_drive_environment
    drive.mount('/content/gdrive')
    !ln -s /content/gdrive/My\ Drive/ /mydrive
    !ls /mydrive

## Git clone Darknet Project

In [ ]:
!git clone https://github.com/AlexeyAB/darknet

## Compile darknet using GPU

In [ ]:
# change makefile to have GPU and OPENCV enabled
%cd darknet
!sed -i 's/OPENCV=0/OPENCV=1/' Makefile
!sed -i 's/GPU=0/GPU=1/' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/' Makefile
!make

## Creating yolo training model 

In [ ]:
!cp cfg/yolov3.cfg cfg/yolov3_training.cfg

In [ ]:
ls /mydrive/yolov2_logos/

## Calculate filter number 

In [3]:
n_classes = 27
number_filters = (n_classes + 5) * 3
print('Total filters: ', number_filters)

Total filters:  96


In [ ]:
!sed -i 's/batch=1/batch=64/' cfg/yolov3_training.cfg
!sed -i 's/subdivisions=1/subdivisions=16/' cfg/yolov3_training.cfg
!sed -i 's/max_batches = 500200/max_batches = 5000/' cfg/yolov3_training.cfg
!sed -i '610 s@classes=80@classes=27@' cfg/yolov3_training.cfg
!sed -i '696 s@classes=80@classes=27@' cfg/yolov3_training.cfg
!sed -i '783 s@classes=80@classes=27@' cfg/yolov3_training.cfg
!sed -i '603 s@filters=255@filters=96@' cfg/yolov3_training.cfg
!sed -i '689 s@filters=255@filters=96@' cfg/yolov3_training.cfg
!sed -i '776 s@filters=255@filters=96@' cfg/yolov3_training.cfg

In [ ]:
!echo -e 'classes=27\ntrain  = data/train.txt\nvalid  = data/test.txt\nnames = data/obj.names\nbackup = /mydrive/yolov2_logos' > data/obj.data
!mkdir data/obj

## Download weights darknet - Transfer Learning

In [ ]:
!wget https://pjreddie.com/media/files/darknet53.conv.74

In [ ]:
!cp /mydrive/yolov2_logos/logos.names ./data/obj.names
!unzip /mydrive/yolov2_logos/images_2.zip -d data/obj

## Creating files 

In [ ]:
import glob

images_list = glob.glob("data/obj/*.jpg")
print(images_list)

file = open("data/train.txt", "w") 
file.write("\n".join(images_list)) 
file.close() 

file = open("data/train.txt", "r") 
for current in file:
    print(current)
    
file = open("data/obj.names", "r") 
for current in file:
    print(current)

## Training object detection

In [ ]:
# Start the training
!sudo ./darknet detector train data/obj.data cfg/yolov3_training.cfg darknet53.conv.74 -dont_show 

## Continue training model

In [ ]:
!sudo ./darknet detector train data/obj.data yolo-obj.cfg backup/yolo-obj_last.weights